In [1]:
from openai import OpenAI

def get_response(base64_image):
    client = OpenAI(
        api_key="sk-a35b6226861c459eaba77cb4e175677b",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    completion = client.chat.completions.create(
        model="qwen2.5-vl-7b-instruct",
        messages=[
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant."}],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpg;base64,{base64_image}"},
                    },
                    {"type": "text", "text": "判断图像是否由AI生成，是则输出“1”，否则输出“0”，不要输出多余内容"},
                ],
            },
        ],
        temperature=0.1,
    )

    return completion.choices[0].message.content

In [2]:
import base64


#  base 64 编码格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [3]:
from tqdm import tqdm
import pandas as pd
test_df=pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/test.csv")
output=pd.DataFrame({'id':[],'label':[]})
for i in tqdm(range(len(test_df))):
    img_path="/kaggle/input/ai-vs-human-generated-dataset/"
    img_path+=test_df.iloc[i]['id']
    base64_image = encode_image(img_path)
    t={'id':test_df.iloc[i]['id'],'label':get_response(base64_image)}
    output.loc[i]=t


100%|██████████| 5540/5540 [7:31:57<00:00,  4.89s/it]


In [4]:
output.to_csv("output_multimodal.csv")